### Goals
- train CNN on my own images + data augmentation
- Fine tune pre-trained model and compare metrics

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
def build_model(model_arch, input_shape=(180, 180, 3), augment=False):
    '''
    Return a keras model with input architecture
    Args:
        model_arch: dict with no of filters, kernal, and pool size for each layer
        input_shape: 
        Augment: bool to add data augmentation or not.
    
    '''
    
    inputs = keras.Input(shape=(input_shape))
    
    if augment:
        data_aug = keras.Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.2),
        ])
        out = data_aug(inputs)
    else:
        out = inputs
    
    out = layers.Rescaling(1./255)(out)
    
    for layer in model_arch.keys():
        filters = model_arch[layer]['filters']
        k_size = model_arch[layer]['k_size']
        p_size = model_arch[layer]['p_size']
        out = layers.Conv2D(filters=filters, kernel_size=k_size, activation='relu')(out)
        out = layers.MaxPooling2D(pool_size=p_size)(out)
        
    # always end with convolution and dense   
    out = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(out)
    x = layers.Flatten()(out)
    
    # for augmented data use dropout to prevent overfitting
    if augment:
        x = layers.Dropout(0.5)(x)
    
    outputs=layers.Dense(3, activation="softmax")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model
    
    

In [3]:
model_arch = {'l2': {'filters': 32, 'k_size': 3, 'p_size': 2},
              'l3': {'filters': 64, 'k_size': 3, 'p_size': 2},
              'l4': {'filters': 128,'k_size': 3, 'p_size': 2},
              'l5': {'filters': 256, 'k_size': 3, 'p_size': 2},
#               'l6': {'filters': 256, 'k_size': 3, 'p_size': 2},
             }

In [4]:
from tensorflow.keras.utils import image_dataset_from_directory

In [5]:
data_dir = './dataset'
train_dir = f'{data_dir}/training_set'
test_dir = f'{data_dir}/test_set'

In [6]:
image_size = (180, 180)
batch_size = 32

In [7]:
train_data = image_dataset_from_directory(train_dir, label_mode='categorical', image_size=image_size, batch_size=batch_size)
test_data = image_dataset_from_directory(test_dir, label_mode='categorical', image_size=image_size, batch_size=batch_size)

Found 1917 files belonging to 3 classes.
Found 477 files belonging to 3 classes.


In [8]:
1917+477

2394

In [9]:
for data_batch, labels_batch in train_data:
    print(f'data batch shape: {data_batch.shape}')
    print(f'labels batch shape: {labels_batch.shape}')
    break

data batch shape: (32, 180, 180, 3)
labels batch shape: (32, 3)


## Vanilla CNN without data augmentation

In [10]:
vanila_model = build_model(model_arch, input_shape=(180, 180, 3), augment=False)

In [11]:
vanila_model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"]
)

In [12]:
vanila_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 128)       73856 

In [13]:
vanila_callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='vanila_model.keras',
        save_best_only=True,
         monitor='val_loss'
    )    
]

In [14]:
vanila_history = vanila_model.fit(
 train_data,
    epochs=20,
    validation_data=test_data,
    callbacks=vanila_callbacks
)

Epoch 1/20
60/60 [==============================] - 225s 4s/step - loss: 1.1052 - accuracy: 0.4163 - val_loss: 0.9455 - val_accuracy: 0.5493
Epoch 2/20
60/60 [==============================] - 232s 4s/step - loss: 1.0068 - accuracy: 0.5456 - val_loss: 0.8559 - val_accuracy: 0.6017
Epoch 3/20
60/60 [==============================] - 233s 4s/step - loss: 0.9148 - accuracy: 0.5676 - val_loss: 0.7778 - val_accuracy: 0.6289
Epoch 4/20
60/60 [==============================] - 232s 4s/step - loss: 0.8876 - accuracy: 0.5989 - val_loss: 0.9736 - val_accuracy: 0.4927
Epoch 5/20
60/60 [==============================] - 232s 4s/step - loss: 0.7404 - accuracy: 0.6813 - val_loss: 0.6910 - val_accuracy: 0.6834
Epoch 6/20
60/60 [==============================] - 233s 4s/step - loss: 0.7231 - accuracy: 0.7011 - val_loss: 1.0485 - val_accuracy: 0.5157
Epoch 7/20
60/60 [==============================] - 234s 4s/step - loss: 0.6360 - accuracy: 0.7324 - val_loss: 0.7074 - val_accuracy: 0.7170
Epoch 8/20
60

#### Model is severly overfit

## Model with data augmentation and dropout

In [15]:
aug_model = build_model(model_arch, input_shape=(180, 180, 3), augment=True)

In [16]:
aug_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 180, 180, 3)       0         
_________________________________________________________________
rescaling_1 (Rescaling)      (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 43, 43, 64)        0   

In [17]:
aug_model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"]
)

In [18]:
aug_callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='data_aug_model.keras',
        save_best_only=True,
         monitor='val_loss'
    )    
]

In [19]:
aug_history = aug_model.fit(
 train_data,
    epochs=20,
    validation_data=test_data,
    callbacks=aug_callbacks
)

Epoch 1/20
60/60 [==============================] - 235s 4s/step - loss: 1.1998 - accuracy: 0.3792 - val_loss: 1.0578 - val_accuracy: 0.4528
Epoch 2/20
60/60 [==============================] - 234s 4s/step - loss: 1.1151 - accuracy: 0.4356 - val_loss: 1.0269 - val_accuracy: 0.4675
Epoch 3/20
60/60 [==============================] - 233s 4s/step - loss: 1.0515 - accuracy: 0.4872 - val_loss: 0.9663 - val_accuracy: 0.5472
Epoch 4/20
60/60 [==============================] - 230s 4s/step - loss: 0.9982 - accuracy: 0.5352 - val_loss: 0.9176 - val_accuracy: 0.5556
Epoch 5/20
60/60 [==============================] - 230s 4s/step - loss: 0.9608 - accuracy: 0.5623 - val_loss: 0.8995 - val_accuracy: 0.5493
Epoch 6/20
60/60 [==============================] - 229s 4s/step - loss: 0.9288 - accuracy: 0.5686 - val_loss: 0.9085 - val_accuracy: 0.5556
Epoch 7/20
60/60 [==============================] - 222s 4s/step - loss: 0.9011 - accuracy: 0.6020 - val_loss: 0.8464 - val_accuracy: 0.6143
Epoch 8/20
60

#### Less severe overfitting

## Feature extraction from pre-trained model
- no data augmentation
- VGG16 model: Use only convolution layers (convolutional base)

In [20]:
conv_base = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False,
    input_shape=(180,180,3)
)

58900480/58889256 [==============================] - 36s 1us/step


In [21]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 180, 180, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 180, 180, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 90, 90, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 90, 90, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 90, 90, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 45, 45, 128)       0     

In [22]:
import numpy as np

In [23]:
def get_features_labels(conv_base, 
                        dataset,
                        preproc=keras.applications.vgg16.preprocess_input, 
                        ):
    features, labels = [], []
    
    for img, label in dataset:
        img_proc = preproc(img)
        feat = conv_base.predict(img_proc)
        features.append(feat)
        labels.append(label)
        
    return np.concatenate(features), np.concatenate(labels)

In [24]:
train_features, train_labels = get_features_labels(conv_base, train_data)
val_features, val_labels = get_features_labels(conv_base, test_data)

In [25]:
inputs = keras.Input(shape=(5,5,512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
pre_model = keras.Model(inputs, outputs)

pre_model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"]
)
pre_callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='feature_extraction_model.keras',
        save_best_only=True,
         monitor='val_loss'
    )    
]

In [26]:
pre_history = pre_model.fit(
 train_features, train_labels,
    epochs=20,
    validation_data=(val_features, val_labels),
    callbacks=pre_callbacks
)

Epoch 1/20
60/60 [==============================] - 4s 49ms/step - loss: 37.4633 - accuracy: 0.8492 - val_loss: 48.6045 - val_accuracy: 0.8176
Epoch 2/20
60/60 [==============================] - 3s 46ms/step - loss: 13.7522 - accuracy: 0.9249 - val_loss: 25.9691 - val_accuracy: 0.8889
Epoch 3/20
60/60 [==============================] - 3s 46ms/step - loss: 4.8065 - accuracy: 0.9656 - val_loss: 29.3873 - val_accuracy: 0.8931
Epoch 4/20
60/60 [==============================] - 3s 47ms/step - loss: 4.0321 - accuracy: 0.9682 - val_loss: 20.5796 - val_accuracy: 0.9099
Epoch 5/20
60/60 [==============================] - 3s 48ms/step - loss: 4.4934 - accuracy: 0.9781 - val_loss: 24.3533 - val_accuracy: 0.9119
Epoch 6/20
60/60 [==============================] - 3s 47ms/step - loss: 2.2831 - accuracy: 0.9844 - val_loss: 23.7965 - val_accuracy: 0.9140
Epoch 7/20
60/60 [==============================] - 3s 47ms/step - loss: 1.6031 - accuracy: 0.9849 - val_loss: 26.5460 - val_accuracy: 0.9182
Epoc

Higher train accuracy and less overfitting